In [21]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
import json
import os

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/matic/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/matic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
folders = [ f.path for f in os.scandir("../../results") if f.is_dir() ]

for fname in folders:

    DATADIR     = fname + "/"
    PREDICTIONS = DATADIR + "predictions.json"
    REFERENCES  = DATADIR + "references.json"

    try:
        predictions = open(PREDICTIONS)
    except:
        continue
    references  = open(REFERENCES)

    pjson = json.load(predictions)
    rjson = json.load(references)

    similarities = []
    recalls = []
    precisions = []
    f1s = []
    errors = 0

    for i in range(len(pjson)):
        pred = pjson[i]['prediction_text']
        refs  = rjson[i]['answers']['text']

        X_list = word_tokenize(pred)

        sim = 0

        for ref in refs:
            #print("Prediction: ", pred)
            #print("Reference: ", ref)

            Y_list = word_tokenize(ref)
            
            # sw contains the list of stopwords
            sw = stopwords.words('slovene') 
            l1 =[];l2 =[]
            
            # remove stop words from the string
            X_set = {w for w in X_list if not w in sw} 
            Y_set = {w for w in Y_list if not w in sw}
            
            # form a set containing keywords of both strings 
            rvector = X_set.union(Y_set) 
            for w in rvector:
                if w in X_set: l1.append(1) # create a vector
                else: l1.append(0)
                if w in Y_set: l2.append(1)
                else: l2.append(0)
            c = 0
            
            # cosine formula 
            for i in range(len(rvector)):
                    c+= l1[i]*l2[i]
            if float((sum(l1)*sum(l2))**0.5) == 0:
                errors += 1
                break
            cosine = c / float((sum(l1)*sum(l2))**0.5)
            #print("similarity: ", cosine)

            if cosine > sim:
                sim = cosine

        similarities.append(sim)
            
        #print(pred)
        #print(refs)

    for i in range(len(pjson)):
        pred = pjson[i]['prediction_text']
        refs  = rjson[i]['answers']['text']

        p = word_tokenize(pred)

        prec = 0
        rec = 0
        f = 0

        for ref in refs:
            #print("Prediction: ", pred)
            #print("Reference: ", ref)

            r = word_tokenize(ref)
            
            # sw contains the list of stopwords
            sw = stopwords.words('slovene') 
            l1 =[];l2 =[]
            
            # remove stop words from the string
            pset = {w for w in p if not w in sw} 
            rset = {w for w in r if not w in sw}

            common_tokens = pset & rset

            if len(pset) == 0 or len(rset) == 0:
                if len(pset) == len(rset):
                    prec = 1
                    rec = 1
                    f = 0
                    break
            else:
                precision = len(common_tokens) / len(pset)
                recall = len(common_tokens) / len(rset)

            if precision + recall == 0:
                f1 = 0
            else:
                f1 = 2 * precision * recall / (precision + recall)

            if f1 > f:
                f = f1
                prec = precision
                rec = recall

        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)

    print(fname)
    print("average cosine similarity: ", np.average(np.array(similarities)))
    print("average precision: ", np.average(np.array(precisions)))
    print("average recall: ", np.average(np.array(recalls)))
    print("average f1-score: ", np.average(np.array(f1s)))
    print()


    predictions.close()
    references.close()

../../results/xlm-roberta-large
average cosine similarity:  0.27132635811640327
average precision:  0.2711790404040404
average recall:  0.3447335248085248
average f1-score:  0.234150974025974

../../results/xlm-roberta-base-whole-dataset-3-epochs-best
average cosine similarity:  0.6360413519925228
average precision:  0.6633487098821107
average recall:  0.6835816217060263
average f1-score:  0.5703276573588417

../../results/multilingual-bert-uncased-slo-fixed
average cosine similarity:  0.26720310478404113
average precision:  0.26461127483627483
average recall:  0.35810847763347764
average f1-score:  0.2401334498834499

../../results/xlm-roberta-base-whole-dataset-1-epoch
average cosine similarity:  0.6107070162141742
average precision:  0.6359819537917236
average recall:  0.6615237186340425
average f1-score:  0.546423321703683

